In [ ]:
import os
print(os.getcwd())
import pandas as pd
import sys
import numpy as np
import datetime as dt
import os
import matplotlib.pyplot as plt
import seaborn as sns
from dateutil.relativedelta import relativedelta
sys.path.append("../../analyzer/getData")
import configparser
config = configparser.ConfigParser()
config.read('../../config.properties')
directorio=config.get('ARCHIVOS', 'archivos_h5s_precios')
print(directorio)
import bdStocks
bd= bdStocks.getData()
sectores=None
fechaI="2008-12-01"
fechaF="2022-06-01"
periodoIndice="M"
fechaI=dt.datetime.strptime(fechaI, '%Y-%m-%d')
fechaF=dt.datetime.strptime(fechaF, '%Y-%m-%d')
files=os.listdir(directorio)
h5s= pd.HDFStore(directorio+files[0], "r")
precios=h5s["data"]
h5s.close()
ex=files[0].split("_")[-1].split(".")[0]
precios["exchange"]=ex
exchanges=[ex]
time_range = pd.date_range(fechaI, fechaF,freq=periodoIndice,normalize=True)
time_range=[e.replace(day=1) for e in time_range]
for file in files[1:]:
    h5s=pd.HDFStore(directorio+file, "r")
    dat_aux=h5s["data"]
    ex=file.split("_")[-1].split(".")[0]
    dat_aux["exchange"]=ex
    precios=pd.concat([precios,dat_aux])
    exchanges.append(ex)
precios.set_index("exchange",inplace=True,append=True)
precios["accion"]=precios.index.get_level_values(2)+"_"+precios.index.get_level_values(1)
precios["fecha"]=precios.index.get_level_values(0)
sectores=bd.getSectors()

In [ ]:
resultados=bd.executeQueryDataFrame("select * from ratios_results where report_date is not null order by report_date asc",None)

In [ ]:
resultados["accion"]=resultados["exchange"]+"_"+resultados["stock"]
resultados.set_index(["fecha","stock","exchange"],inplace=True)


In [ ]:
indice=bd.executeQueryDataFrame("select * from preciosindices where indice='SP500' ORDER BY date  ASC",None)
indice["Date"]=pd.to_datetime(indice.Date)
indice.set_index("Date",inplace=True)

In [ ]:
precios_stock_ano=precios.loc[precios.fecha<dt.datetime(2022,1,1)]
precios_stock_year_to_year=precios.loc[precios.fecha<dt.datetime.today()-relativedelta(years=1)]
precios_stock_2020=precios.loc[precios.fecha<dt.datetime(2020,1,1)]
precios_stock_2015=precios.loc[precios.fecha<dt.datetime(2015,1,1)]
precios_stock_2010=precios.loc[precios.fecha<dt.datetime(2010,1,1)]
times=["actual","year","yoy","2020","2015","2010"]

In [ ]:
sectors=bd.executeQuery("select exchange,stock,sector from sectors" ,params=None)
sectors={e[0]+"_"+e[1]:e[2] for e in sectors}

highlits=bd.executeQueryDataFrame("select * from highlights" ,params=None)
highlits["accion"]=highlits.exchange+"_"+highlits.stock
highlits=highlits.groupby("accion",as_index=False).last()

In [ ]:
aux=precios.groupby("accion").last()
highlits["close"]=0
highlits["close"]=highlits["accion"].transform(lambda x:aux.loc[x].Adjusted_close if x in   aux.index else 0)
highlits=highlits.loc[highlits.exchange.isin(["US","LSE","XETRA","MI","PA","TO","MC"])]
highlits["close"]=highlits.apply(lambda x:x.close*0.01161 if x.exchange=="LSE" else x.close,axis=1)

In [ ]:
consulta2=("select CONCAT(exchange,'_',company) as accion from indices inner join exchangeIndice on exchangeIndice.indice=indices.indice where indices.indice like '%nasda%';")
stocks2=bd.executeQueryDataFrame(consulta2,None)
stocks_total=stocks2.accion.values


In [ ]:

highlits

In [ ]:

pd.set_option('display.max_rows', 500)
highlits["prizeToBook"]=highlits.close/highlits.BookValue
highlits.loc[highlits.accion.isin(stocks_total)].\
loc[:,["accion","fecha","MarketCapitalization","PERatio","prizeToBook"]].sort_values(by="MarketCapitalization",ascending=False)
  

In [ ]:
import seaborn as sns
for e in ["MarketCapitalization","PERatio","PEGRatio","BookValue"]:
    aux=highlits.sort_values(by=e)
    aux=aux.loc[:,[e]]
    aux=np.log10(aux)
    plt.figure(figsize=(10,10))
    #plt.hist(aux.loc[:,e])
    plt.boxplot(aux.loc[:,e].dropna())
    plt.title(e)
import math
print(math.pow(10,1.2))

<h2>Analisis de empresas de VPN y ciberseguridad.

In [ ]:
palabra_clave="%cybersecurity%"

In [ ]:
descripciones=bd.executeQuery("select exchange,stock,description from descriptions where description like '%cyber security%'\
or description like %s",params=(palabra_clave,))
descripciones={e[0]+"_"+e[1]:e[2] for e in descripciones}
descripciones
itera=iter(list(descripciones.items()))

In [ ]:
highlits.loc[highlits.accion.isin(descripciones.keys())].\
loc[:,["accion","fecha","MarketCapitalization","PERatio","prizeToBook","close"]].sort_values(by="prizeToBook",ascending=False)

In [ ]:
next(itera)

In [ ]:
precios1=precios.loc[precios.accion.isin(descripciones.keys())]
precios_stock_ano=precios1.loc[precios1.fecha<dt.datetime(2022,1,1)]
precios_stock_year_to_year=precios1.loc[precios1.fecha<dt.datetime.today()-relativedelta(years=1)]
precios_stock_2020=precios1.loc[precios1.fecha<dt.datetime(2020,1,1)]
precios_stock_2015=precios1.loc[precios1.fecha<dt.datetime(2015,1,1)]
precios_stock_2010=precios1.loc[precios1.fecha<dt.datetime(2010,1,1)]
times=["actual","year","yoy","2020","2015","2010"]

In [ ]:
a=[]
from functools import reduce
for i,u in zip(times,[precios1,precios_stock_ano,\
          precios_stock_year_to_year,\
          precios_stock_2020,precios_stock_2015,
          precios_stock_2010]):
    aux=u.groupby("accion").last()
    aux.drop("fecha",inplace=True,axis=1)
    aux.columns=[i]
    a.append(aux)
    
df = reduce(
                lambda left,right: pd.merge(
                    left,
                    right,
                    left_index=True, 
                    right_index=True, 
                    how='outer',
                    suffixes=('', '_drop')
                ), 
                a
            )

In [ ]:
for e in df.columns:
    if e!="actual":
        df[e]=(df.actual-df[e])/df[e]

In [ ]:
df_pct_change=df.sort_values(by="year",ascending=False)
df_pct_change.head(60)

In [ ]:
df_pct_change.year.mean()

In [ ]:
stocks=(list(df_pct_change.index))


In [ ]:
stocks=["US_RPD","US_LDOS","US_NLOK","US_TENB","LSE_NCC","LSE_KAPE","MI_XPR","US_FTNT","US_SPLK","US_CYBR"]
stocks=iter(stocks)

In [ ]:
stock=next(stocks)
f_aux=resultados.loc[resultados.accion==stock]
f_aux.loc[f_aux.index[-1],"adjusted_close"]=precios.loc[precios.accion==stock].iloc[-1,0]
f_aux["per"]=f_aux.adjusted_close*f_aux.commonStockSharesOutstanding/f_aux.netIncome/4
p_aux=precios.loc[precios.accion==stock].droplevel(["stock","exchange"])

In [ ]:
descripciones[stock],df_pct_change.loc[stock]

In [ ]:
p_aux.tail(60)

In [ ]:
p_aux["indice"]=indice["Close"]
p_aux=p_aux.loc[p_aux.index>dt.datetime(2000,1,1)].dropna()
plt.plot(p_aux.index.get_level_values(0),(p_aux.Adjusted_close-p_aux.Adjusted_close[0])/p_aux.Adjusted_close[0])
plt.plot(p_aux.index.get_level_values(0),(p_aux.indice-p_aux.indice[0])/p_aux.indice[0])
plt.xticks(rotation = 'vertical')
plt.legend([stock,"indice"])
u=p_aux.loc[:,["indice","Adjusted_close"]]
u["fecha"]=u.index
#u=u.groupby([u.fecha.dt.year,u.fecha.dt.month]).head(1)
corr=u.corr()
corr



In [ ]:
f_aux.loc[:,["netIncome","adjusted_close","earnings","totalAssets","ebit"]].corr()

In [ ]:
f_aux.loc[f_aux.index[-1],"adjusted_close"]=p_aux.Adjusted_close[-1]
f_aux["fecha"]=pd.to_datetime(f_aux.index.get_level_values(0))
f_aux=f_aux.loc[f_aux.fecha>dt.datetime(2012,1,1)]
f_aux["per"]=(f_aux.adjusted_close*f_aux.commonStockSharesOutstanding)/f_aux.netIncome
print(list(f_aux.per)[-1]
      /4,list(f_aux.earnings_per)[-1]/4)
c=[]
for columna in f_aux.columns:
    if f_aux[columna].dtypes==float:
        if f_aux.isna().sum()[columna]<f_aux.shape[0]/2:
            c.append(columna)
figure,ax=plt.subplots(int(len(c)/2)+1,2,figsize=(20,40))
i=j=0
for u in c:
    ax[i,j].plot(f_aux["fecha"],f_aux[u],marker="o")
    ax[i,j].title.set_text(stock+"_"+u)
    j+=1
    if j==2:
        j=0
        i+=1

Seleccionamos las empresas: US_SPLK,US_FTNT,LSE_KAPE,LSE_NCC,US_RPD,XETRA_YSN,LSE_SPT basandonos en su crecimiento los ultimos dos años, ultimas hojas de resultados y precio actual.

# Hojas de resultados:
* US_SPLK: https://investors.splunk.com/financial-information/quarterly-results
* US_FTNT: https://investor.fortinet.com/quarterly-earnings/
* LSE_KAPE: https://investors.kape.com/investors/reports-presentations
* LSE_NCC: https://www.nccgroupplc.com/investor-relations/results-media/
* US_RPD: https://investors.rapid7.com/financial-information/annual-reports
* XETRA_YSN: https://www.secunet.com/ueber-uns/investoren
* LSE_SPT: https://corporate.spirent.com/investors/results




Analisis de portfolio optimo basandonos en los tres ultimos años.

In [ ]:
columnas=["US_SPLK","US_FTNT","LSE_KAPE","LSE_NCC","US_RPD","XETRA_YSN","LSE_SPT"]
semi=precios.loc[precios.accion.isin(columnas)]
semi=semi.loc[semi.fecha>dt.datetime(2018,1,1)]
prec=pd.DataFrame()
prec.index=semi.droplevel(["stock","exchange"]).index
for e in columnas:
    prec[e]=semi.loc[semi.accion==e].droplevel(["stock","exchange"],axis=0).Adjusted_close

In [ ]:
fecha1=dt.datetime(2010,6,1)
fecha2=dt.datetime(2022,6,1)
precios_diarios=pd.DataFrame(index=pd.date_range(fecha1,fecha2,freq="B"))
consulta = " select fecha,adjusted_close from {}_precios where stock=%s and fecha>%s and fecha<%s"
for e in columnas:
    ar=e.split("_")
    ex=ar[0]
    st=ar[1]
    data=bd.executeQueryDataFrame(consulta.format(ex),(st,fecha1,fecha2))
    data.index=pd.to_datetime(data.fecha)
    precios_diarios[e]=data.adjusted_close
                    
precios_diarios.tail()
prec=precios_diarios
precios_diarios.tail()

In [ ]:
precios_diarios_transformed=pd.DataFrame(index=range(1,360))
precios_diarios_transformed["US_SPLK"]=precios_diarios[-950:-590].reset_index()["US_SPLK"]
precios_diarios_transformed["US_FTNT"]=precios_diarios[-560:-200].reset_index()["US_FTNT"]
precios_diarios_transformed["LSE_KAPE"]=precios_diarios[-560:-200].reset_index()["LSE_KAPE"]
precios_diarios_transformed["LSE_NCC"]=precios_diarios[-860:-400].reset_index()["LSE_NCC"]

for e in ["US_RPD","XETRA_YSN","LSE_SPT"]:
    precios_diarios_transformed[e]=precios_diarios[-560:-200].reset_index()[e]
precios_diarios_transformed["LSE_SPT"]=precios_diarios[-860:-400].reset_index()["LSE_SPT"]
prec=precios_diarios_transformed

In [ ]:
precios_diarios.tail(30)

In [ ]:
prec_change=prec.pct_change()
covarianzas=prec_change.cov()
descuento=0.1/12
medias=prec_change.mean()
medias,np.dot(np.linalg.inv(covarianzas),np.array(medias))

In [ ]:
precios_hoy=precios_diarios.loc[dt.datetime.strptime("2022-05-09","%Y-%m-%d")]
pesos=np.dot(np.linalg.inv(covarianzas),np.array(medias))
precios_hoy["US_RPD"]=63
2400*pesos/sum(pesos)/precios_hoy/0.01161

In [ ]:
plt.figure(figsize=(15,15))
leyenda=[]
for r in precios_diarios.columns:
    a=precios_diarios[r]
    plt.plot(precios_diarios.index,(a-a.mean())/a.std())
    leyenda.append(r)
plt.legend(leyenda)    

In [ ]:
from scipy.stats import shapiro
from statsmodels.stats.diagnostic import lilliefors
for k in prec_log.columns:
    plt.figure(figsize=(10,10))
    plt.hist(prec_log[k],bins=20)
    plt.title(k+ "log returns")
    print(k+", shapiro:"+str(shapiro(prec_log[k].dropna())))
    print(k+", lilliefors:"+str(lilliefors(list(prec_log[k].dropna()))))
    plt.figure(figsize=(10,10))
    plt.hist(prec_change[k],bins=20)
    plt.title(k+ "net returns")

In [ ]:
prec_log=np.log(prec).diff(1)
cov_log=prec_log.cov()
mean_log=prec_log.mean()
suma=(abs(mean_log-(medias- np.diag(covarianzas)/2)))/(abs(mean_log))
mean_log,medias- np.diag(covarianzas)/2,suma

<h2>Analisis de empresas de energias

In [ ]:
palabra_clave="%crude oil%"
palabra_clave2="%natural gas%"
sectors=bd.executeQuery("select exchange,stock,sector from sectors" ,params=None)
sectors={e[0]+"_"+e[1]:e[2] for e in sectors}
sectors=bd.executeQuery("select exchange,stock,sector from sectors" ,params=None)
sectors={e[0]+"_"+e[1]:e[2] for e in sectors}
descripciones=bd.executeQuery("select exchange,stock,description from descriptions where description like %s \
or description like %s",params=(palabra_clave,palabra_clave2))
descripciones={e[0]+"_"+e[1]:e[2] for e in descripciones}


In [ ]:
itera=iter(list(descripciones.items()))
len(descripciones)

In [ ]:

next(itera)

In [ ]:
precios1=precios.loc[precios["accion"].isin(descripciones.keys())]
precios_stock_ano=precios1.loc[precios1.fecha<dt.datetime(2022,1,1)]
precios_stock_year_to_year=precios1.loc[precios1.fecha<dt.datetime.today()-relativedelta(years=1)]
precios_stock_2020=precios1.loc[precios1.fecha<dt.datetime(2020,1,1)]
precios_stock_2015=precios1.loc[precios1.fecha<dt.datetime(2015,1,1)]
precios_stock_2010=precios1.loc[precios1.fecha<dt.datetime(2010,1,1)]
times=["actual","year","yoy","2020","2015","2010"]
a=[]
from functools import reduce
for i,u in zip(times,[precios,precios_stock_ano,\
          precios_stock_year_to_year,\
          precios_stock_2020,precios_stock_2015,
          precios_stock_2010]):
    aux=u.groupby("accion").last()
    aux.drop("fecha",inplace=True,axis=1)
    aux.columns=[i]
    a.append(aux)
    
df = reduce(
                lambda left,right: pd.merge(
                    left,
                    right,
                    left_index=True, 
                    right_index=True, 
                    how='outer',
                    suffixes=('', '_drop')
                ), 
                a
            )
for e in df.columns:
    if e!="actual":
        df[e]=(df.actual-df[e])/df[e]
df_pct_change=df.sort_values(by="year",ascending=False)
df_pct_change.tail(60)
df_pct_change.year.mean()
stocks=iter(df_pct_change.index)

In [ ]:
df=df.loc[df.index.isin(descripciones.keys())]
df.sort_values("year",ascending=False),df.shape
df.year.mean()

In [ ]:
df.sort_values("year",ascending=False).head(50)

In [ ]:
admiral=bd.executeQueryDataFrame\
("select CONCAT(exchange,'_',code) as accion from admiralmarkets",None)
admiral

In [ ]:
high=highlits.loc[(highlits.accion.isin(descripciones.keys()))&(highlits.accion.isin(list(admiral.accion)))]
#high=highlits.loc[(highlits.accion.isin(descripciones.keys()))]
high.index=high.accion
high.shape,highlits.shape

In [ ]:
total=pd.merge(high,df,how='inner',left_index=True,right_index=True)
total=total.loc[total.MarketCapitalization>1000000000]
#total=total.drop(["accion","stock","fecha","exchange","MostRecentQuarter"],axis=1)
total.shape

In [ ]:
total.columns

In [ ]:
import seaborn as sns
total.corr()


harvest = np.array(round(df.corr(),3))
columns=["MarketCapitalization",\
              "PERatio","prizeToBook","RevenueTTM",\
              "DividendShare","QuarterlyRevenueGrowthYOY",\
              "QuarterlyEarningsGrowthYOY","year","2020","2015"]
df_numerico1=total.loc[:,columns]
vegetables = df_numerico1.columns
cor=np.array(round(df_numerico1.corr(),3))
farmers = df_numerico1.columns
fig, ax = plt.subplots(figsize = (10,10))
ax.pcolormesh(np.arange(len(df_numerico1.columns)+1), np.arange(len(df_numerico1.columns)+1), df_numerico1.corr().values, cmap = 'Blues')
plt.xticks(np.arange(len(df_numerico1.columns)),df_numerico1.columns, rotation = 'vertical');
plt.yticks(np.arange(len(df_numerico1.columns)), df_numerico1.columns, rotation = 'horizontal');
for i in range(len(vegetables)):
    for j in range(len(farmers)):
        text = ax.text(j, i,cor[i, j],
                       ha="left", va="center", color="r")

In [ ]:
total.groupby("exchange").count()

In [ ]:
for columna in columns:
   
    plt.figure(figsize=(8,8))
    sns.boxplot(x=total.exchange, y=(total[columna]),color="green")

In [ ]:
total.shape

In [ ]:
columns=[\
           "MarketCapitalization",   "fecha","PERatio","prizeToBook","RevenueTTM",
         "QuarterlyRevenueGrowthYOY","QuarterlyEarningsGrowthYOY",
        "year","2020","2015","close"]
total["revenue/price"]=total.QuarterlyRevenueGrowthYOY/total.year
total.loc[:,columns+["revenue/price"]].sort_values(by="revenue/price",ascending=False)

In [ ]:
stock="TO_TRP"
descripciones[stock]

In [ ]:
total.isna().sum(),total.shape

In [ ]:
pd.set_option('display.max_rows', 500)
total.loc[:,["year","PERatio","prizeToBook","RevenueTTM","DividendShare","QuarterlyRevenueGrowthYOY","QuarterlyEarningsGrowthYOY"]].sort_values(by="year",ascending=False)

In [ ]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
columns=["MarketCapitalization",\
              "PERatio","prizeToBook","RevenueTTM",\
              "DividendShare","QuarterlyRevenueGrowthYOY",\
              "QuarterlyEarningsGrowthYOY"]
standar_scaler=StandardScaler()
X=total.dropna()
X=standar_scaler.fit_transform(X)
pca=PCA(n_components=7)
pca.fit(X)

compoentes=pd.DataFrame(np.transpose(pca.components_),index=total.columns)
compoentes,np.cumsum(pca.explained_variance_ratio_)

In [ ]:
t=total.sort_values(by="PERatio").loc[:,["MarketCapitalization","PERatio","actual","year","2020","prizeToBook","BookValue","close"]]
stocks=iter(total.sort_values(by="revenue/price").loc[:,["MarketCapitalization","PERatio","actual","year","2020","prizeToBook"]].index)


In [ ]:

f_aux=resultados.loc[resultados.accion==stock]
f_aux.loc[f_aux.index[-1],"adjusted_close"]=precios.loc[precios.accion==stock].iloc[-1,0]
f_aux["per"]=f_aux.adjusted_close*f_aux.commonStockSharesOutstanding/f_aux.netIncome/4
p_aux=precios.loc[precios.accion==stock].droplevel(["stock","exchange"])

In [ ]:
f_aux.loc[f_aux.index[-1],"adjusted_close"]=p_aux.Adjusted_close[-1]
f_aux["fecha"]=pd.to_datetime(f_aux.index.get_level_values(0))
f_aux=f_aux.loc[f_aux.fecha>dt.datetime(2012,1,1)]
f_aux["per"]=(f_aux.adjusted_close*f_aux.commonStockSharesOutstanding)/f_aux.netIncome
print(list(f_aux.per)[-1]
      /4,list(f_aux.earnings_per)[-1]/4)
c=[]
for columna in f_aux.columns:
    if f_aux[columna].dtypes==float:
        if f_aux.isna().sum()[columna]<f_aux.shape[0]/2:
            c.append(columna)
figure,ax=plt.subplots(int(len(c)/2)+1,2,figsize=(20,40))
i=j=0
for u in c:
    ax[i,j].plot(f_aux["fecha"],f_aux[u],marker="o")
    ax[i,j].title.set_text(stock+"_"+u)
    j+=1
    if j==2:
        j=0
        i+=1

In [ ]:
f_aux1=f_aux.droplevel(["stock","exchange"])
f_aux1.index=pd.to_datetime(f_aux1.index)
f_aux1.resample("y").sum()
f_aux1["per"]=f_aux1.adjusted_close*f_aux1.commonStockSharesOutstanding/(4*f_aux1.netIncome)
f_aux1.per


In [ ]:
fecha1=dt.datetime(2003,1,1)
fecha2=dt.datetime(2022,5,1)
precios_diarios=pd.DataFrame(index=pd.date_range(fecha1,fecha2,freq="B"))
consulta = " select fecha,adjusted_close from {}_precios where stock=%s and fecha>%s and fecha<%s"
for e in total.index:
    ar=e.split("_")
    ex=ar[0]
    st=ar[1]
    data=bd.executeQueryDataFrame(consulta.format(ex),(st,fecha1,fecha2))
    data.index=pd.to_datetime(data.fecha)
    precios_diarios[e]=data.adjusted_close
                    
precios_diarios.tail()
prec=precios_diarios
precios_diarios.tail()

In [105]:
precios_diarios.isna().sum(),precios_diarios.shape
crude_oil=pd.read_csv("/media/manuel/TRANSCEND/dataBolsa/tablas_varias_downloaded/crude_oil.csv")

In [106]:
crude_oil["Fecha"]=crude_oil["Fecha"].transform(lambda x:dt.datetime.strptime(x,"%d.%m.%Y"))
crude_oil.set_index("Fecha",inplace=True)
crude_oil.sort_index(inplace=True)
crude_oil.tail(10)


,Último,Apertura,Máximo,Mínimo,Vol.,% var.
Fecha,,,,,,
2022-05-03,"102,41","105,08","105,80","102,10","239,27K","-2,62%"
2022-05-04,"107,81","103,50","108,61","102,95","272,91K","5,27%"
2022-05-05,"108,26","107,58","111,37","106,45","286,92K","0,42%"
2022-05-06,"109,77","108,70","111,18","107,24","294,28K","1,39%"
2022-05-08,"109,17","110,47","110,47","108,81",-,"-0,55%"
2022-05-09,"103,09","110,43","110,49","102,13","368,18K","-5,57%"
2022-05-10,"99,76","102,65","104,16","98,86","389,75K","-3,23%"
2022-05-11,"105,71","99,00","106,44","98,20","382,70K","5,96%"
2022-05-12,"106,13","105,63","107,37","102,66","323,55K","0,40%"


In [107]:
crude_oil.tail(10)

,Último,Apertura,Máximo,Mínimo,Vol.,% var.
Fecha,,,,,,
2022-05-03,"102,41","105,08","105,80","102,10","239,27K","-2,62%"
2022-05-04,"107,81","103,50","108,61","102,95","272,91K","5,27%"
2022-05-05,"108,26","107,58","111,37","106,45","286,92K","0,42%"
2022-05-06,"109,77","108,70","111,18","107,24","294,28K","1,39%"
2022-05-08,"109,17","110,47","110,47","108,81",-,"-0,55%"
2022-05-09,"103,09","110,43","110,49","102,13","368,18K","-5,57%"
2022-05-10,"99,76","102,65","104,16","98,86","389,75K","-3,23%"
2022-05-11,"105,71","99,00","106,44","98,20","382,70K","5,96%"
2022-05-12,"106,13","105,63","107,37","102,66","323,55K","0,40%"


In [108]:
precios_diarios["oil"]=crude_oil["Apertura"].transform(lambda x:x.replace(",",".")).astype(float)
precios_diarios.shape

(5043, 33)

In [ ]:
#precios_diarios=precios_diarios.resample("M").last()

In [ ]:
import seaborn as sns



vegetables =farmers= precios_diarios.corr().columns
cor=np.array(round(precios_diarios.loc[precios_diarios.index>dt.datetime(2022,2,1)].corr(),2))
fig, ax = plt.subplots(figsize = (20,20))
ax.pcolormesh(np.arange(len(precios_diarios.columns)+1), np.arange(len(precios_diarios.columns)+1), cor, cmap = 'Blues')
plt.xticks(np.arange(len(precios_diarios.columns)),precios_diarios.columns, rotation = 'vertical');
plt.yticks(np.arange(len(precios_diarios.columns)), precios_diarios.columns, rotation = 'horizontal');
for i in range(len(vegetables)):
    for j in range(len(farmers)):
        text = ax.text(j, i,cor[i, j],
                       ha="left", va="center", color="r",size=13)
        
cor1=np.array(round(precios_diarios.corr(),2))
fig, ax = plt.subplots(figsize = (20,20))
ax.pcolormesh(np.arange(len(precios_diarios.columns)+1), np.arange(len(precios_diarios.columns)+1), cor1, cmap = 'Blues')
plt.xticks(np.arange(len(precios_diarios.columns)),precios_diarios.columns, rotation = 'vertical');
plt.yticks(np.arange(len(precios_diarios.columns)), precios_diarios.columns, rotation = 'horizontal');
for i in range(len(vegetables)):
    for j in range(len(farmers)):
        text = ax.text(j, i,cor1[i, j],
                       ha="left", va="center", color="r",size=13)

In [ ]:
# 
p=(precios_diarios-precios_diarios.mean())/precios_diarios.std()
p=p.loc[p.index>dt.datetime(2005,1,1)]
p["fecha"]=p.index
for k in precios_diarios.columns:
    if k!="oil":
        fig, ax = plt.subplots(figsize=(15,15))
        ax= sns.lineplot(x='fecha', y=k, data=p,marker="o")
        ax1 = sns.lineplot(x='fecha', y="oil", data=p,marker="o")
       

Empresas seleccionadas para analizar: LSE_HILS, TO_CNQ, TO_CVE, PA_ENGI, MI_TOT, TO_ENB, TO_FNV, XETRA_REP, XETRA_EOAN, XETRA_LIN, TO_INO, ZETRA_CAT1, TO_TRP

In [111]:
stocks=["LSE_HILS", "TO_CNQ",\
        "TO_CVE", "PA_ENGI",\
        "MI_TOT","TO_ENB",\
        "TO_FNV","XETRA_REP",\
        "XETRA_EOAN","XETRA_LIN",\
        "TO_IMO", "XETRA_CAT1",\
        "TO_TRP"]
datos=total.loc[stocks]
precios_diarios=precios_diarios.loc[:,stocks+["oil"]]
datos.loc[:,columns]

,MarketCapitalization,PERatio,prizeToBook,RevenueTTM,DividendShare,QuarterlyRevenueGrowthYOY,QuarterlyEarningsGrowthYOY
accion,,,,,,,
LSE_HILS,1.033165e+09,30.4000,3.573984,7.050000e+08,0.310,0.017,1.649
TO_CNQ,8.853905e+10,11.7895,2.594478,3.005700e+10,1.998,0.836,2.402
TO_CVE,4.892719e+10,25.7724,1.612240,5.326200e+10,0.105,0.743,6.900
PA_ENGI,2.663569e+10,7.5689,0.728641,5.786600e+10,0.850,0.139,-0.963
MI_TOT,1.253749e+11,7.7814,1.134190,2.099510e+11,2.994,0.655,0.504
TO_ENB,1.133152e+11,19.4843,2.231765,4.707100e+10,3.340,0.251,0.035
TO_FNV,3.511014e+10,36.7971,6.160051,1.297600e+09,1.160,0.076,0.255
XETRA_REP,2.003283e+10,6.2298,0.922772,5.179700e+10,0.000,0.722,1.293
XETRA_EOAN,2.466572e+10,5.2585,2.092641,7.811900e+10,0.490,0.654,4.805


In [ ]:
import seaborn as sns



vegetables =farmers= precios_diarios.corr().columns
cor=np.array(round(precios_diarios.loc[precios_diarios.index>dt.datetime(2022,2,1)].corr(),2))
fig, ax = plt.subplots(figsize = (20,20))
ax.pcolormesh(np.arange(len(precios_diarios.columns)+1), np.arange(len(precios_diarios.columns)+1), cor, cmap = 'Blues')
plt.xticks(np.arange(len(precios_diarios.columns)),precios_diarios.columns, rotation = 'vertical');
plt.yticks(np.arange(len(precios_diarios.columns)), precios_diarios.columns, rotation = 'horizontal');
for i in range(len(vegetables)):
    for j in range(len(farmers)):
        text = ax.text(j, i,cor[i, j],
                       ha="left", va="center", color="r",size=13)
        
cor1=np.array(round(precios_diarios.corr(),2))
fig, ax = plt.subplots(figsize = (20,20))
ax.pcolormesh(np.arange(len(precios_diarios.columns)+1), np.arange(len(precios_diarios.columns)+1), cor1, cmap = 'Blues')
plt.xticks(np.arange(len(precios_diarios.columns)),precios_diarios.columns, rotation = 'vertical');
plt.yticks(np.arange(len(precios_diarios.columns)), precios_diarios.columns, rotation = 'horizontal');
for i in range(len(vegetables)):
    for j in range(len(farmers)):
        text = ax.text(j, i,cor1[i, j],
                       ha="left", va="center", color="r",size=13)

In [ ]:
p=p.loc[p.fecha>dt.datetime(2010,1,1)]
for k in precios_diarios.columns:
    
    if k!="oil":
        f_aux=resultados.loc[(resultados.accion==k)&(resultados.index.get_level_values(0)>dt.date(2010,1,1))]
        f_aux["per"]=f_aux.adjusted_close*f_aux.commonStockSharesOutstanding/f_aux.netIncome/4
        f_aux=(f_aux-f_aux.mean())/f_aux.std()
        fig, ax = plt.subplots(figsize=(15,8))
        ax= sns.lineplot(x='fecha', y=p.resample("m").last()[k], data=p,color="g")
        ax1 = sns.lineplot(x='fecha', y=p.resample("m").last().oil, data=p,color="r")
        sns.lineplot(x='fecha', y="netIncome", data=f_aux,color="y")
        sns.lineplot(x='fecha', y="totalRevenue", data=f_aux,color="b")
        sns.lineplot(x='fecha', y="totalAssets", data=f_aux,color="orange")
        plt.legend([k,"oil","netIncome","totalRevenue","totalAssets"])
        plt.ylabel(k)

In [ ]:
f_aux

In [ ]:
f_aux.tail(10)

Guardamos los datos para analizar mas profundamente

In [112]:
precios_diarios.to_csv("../cointegracion/data/raw/dayly_energy.csv")
precios_diarios.shape

(5043, 14)

In [115]:
p=p.loc[p.fecha>dt.datetime(2010,1,1)]
for k in precios_diarios.columns:
 
    if k!="oil":
        f_aux=resultados.loc[(resultados.accion==k)&(resultados.index.get_level_values(0)>dt.date(2010,1,1))]
        f_aux=f_aux.droplevel(["exchange","stock"])
        f_aux.index=pd.to_datetime(f_aux.index)
        print(k)
        f_aux=f_aux.resample("Q").last()
        f_aux["oil"]=precios_diarios.resample("Q").last().oil
        f_aux.to_csv("../cointegracion/data/raw/energy_fundamental_{}.csv".format(k))
    print(k)

LSE_HILS
LSE_HILS
TO_CNQ
TO_CNQ
TO_CVE
TO_CVE
PA_ENGI
PA_ENGI
MI_TOT
MI_TOT
TO_ENB
TO_ENB
TO_FNV
TO_FNV
XETRA_REP
XETRA_REP
XETRA_EOAN
XETRA_EOAN
XETRA_LIN
XETRA_LIN
TO_IMO
TO_IMO
XETRA_CAT1
XETRA_CAT1
TO_TRP
TO_TRP
oil


In [114]:
f_aux.tail(50)

,netIncome,totalAssets,totalLiab,totalCurrentLiabilities,totalStockholderEquity,commonStockSharesOutstanding,netDebt,freeCashFlow,totalRevenue,ebit,...,per,earnings,report_date,earnings_per,prizebook,solvenciaLargo,solvenciaCorto,liquidez,accion,oil
fecha,,,,,,,,,,,,,,,,,,,,,
2010-03-31,3.351014e+08,4.288193e+10,2.678315e+10,5.023757e+09,NaN,980815927.0,1.874295e+10,-5.715950e+08,2.129691e+09,4.620983e+08,...,61.55992,0.48,2010-04-30,43.81729,0.48106,NaN,NaN,-0.11378,TO_TRP,82.50
2010-06-30,3.217341e+08,4.393062e+10,2.740043e+10,4.974171e+09,NaN,980815927.0,1.925930e+10,-5.033959e+08,2.060794e+09,4.353987e+08,...,69.38727,0.40,2010-07-29,56.90225,0.50817,NaN,NaN,-0.10120,TO_TRP,75.46
2010-09-30,3.544897e+08,4.482642e+10,2.798429e+10,4.843780e+09,NaN,980815927.0,1.977654e+10,-5.042639e+08,2.083529e+09,4.702660e+08,...,64.29138,0.54,2010-11-03,43.03037,0.50842,NaN,NaN,-0.10411,TO_TRP,77.91
2010-12-31,3.306597e+08,4.541395e+10,2.822453e+10,5.116188e+09,NaN,980815927.0,2.021803e+10,-5.485095e+08,2.074686e+09,4.501772e+08,...,72.09358,0.55,2011-02-15,44.19036,0.52491,NaN,NaN,-0.10721,TO_TRP,89.67
2011-03-31,3.634399e+08,4.560630e+10,2.812935e+10,5.119458e+09,NaN,980815927.0,2.039668e+10,-2.906721e+08,2.130791e+09,5.144516e+08,...,72.39889,0.61,2011-04-29,43.97918,0.57695,NaN,NaN,-0.05678,TO_TRP,104.31
2011-06-30,3.646266e+08,4.569987e+10,2.788623e+10,4.931305e+09,NaN,980815927.0,2.036246e+10,-1.121143e+08,2.134861e+09,5.209678e+08,...,71.15588,0.51,2011-07-28,51.86824,0.56773,NaN,NaN,-0.02274,TO_TRP,95.08
2011-09-30,3.860844e+08,4.638124e+10,2.823049e+10,5.165204e+09,NaN,980815927.0,2.066297e+10,4.825736e+07,2.220907e+09,5.346452e+08,...,72.04093,0.59,2011-11-01,48.06424,0.59968,NaN,NaN,0.00934,TO_TRP,83.07
2011-12-31,3.980563e+08,4.725250e+10,2.888899e+10,5.403469e+09,NaN,980815927.0,2.094631e+10,-1.449516e+07,2.267271e+09,5.380968e+08,...,67.88800,0.52,2012-02-14,52.98423,0.57189,NaN,NaN,-0.00268,TO_TRP,99.78
2012-03-31,3.990375e+08,4.713333e+10,2.880433e+10,5.174646e+09,NaN,980815927.0,2.095954e+10,5.966995e+07,2.148514e+09,5.353979e+08,...,66.93034,0.52,2012-04-27,52.36558,0.56664,NaN,NaN,0.01153,TO_TRP,103.29


<h2>Analisis de empresas de IBEX35.

<h2>Analisis basandose en situacion macro, correlacion entre empresas y resultados cuatrimestrales de empresas del ibex35.

In [ ]:
indice='ibex35'
consulta="select CONCAT('MC','_',company) as accion from indices where indice=%s"
stocks_ibex35= (bd.executeQueryDataFrame(consulta,(indice,)).accion.values)
precios_ibex35=precios.loc[precios.accion.isin(stocks_ibex35)]


In [ ]:
columnas=np.unique(precios_ibex35.accion)
semi=precios_ibex35.copy()
semi=semi.loc[semi.fecha>dt.datetime(2018,1,1)]
prec_ibex35=pd.DataFrame()
prec_ibex35.index=semi.droplevel(["stock","exchange"]).index
for e in columnas:
    prec_ibex35[e]=semi.loc[semi.accion==e].droplevel(["stock","exchange"],axis=0).Adjusted_close
print(len(columnas))

In [ ]:
for k in precios_diarios.columns:
    

In [ ]:
prec_ibex35.corr()